In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

In [2]:
df=pd.read_csv('Stress posts.csv')

In [3]:
df.columns=['id','text','time']

In [4]:
df.isnull().sum()
df=df.dropna()

In [5]:
df.duplicated().sum()

0

In [6]:
new_df=df.copy()

In [7]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\upend\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
import re

stemmer = PorterStemmer()

# Load English stopwords
english_stopwords = set(stopwords.words("english"))

def preprocess_text(text):
    # Convert text to lowercase
    text = str(text).lower()
    
    # Remove URLs
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    
    # Remove digits and single characters
    text = re.sub(r'\b\w\b|\d+', '', text)
    
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    
    # Tokenize the text and remove stopwords
    words = [word for word in text.split() if word not in english_stopwords]
    
    # Stem the words
    stemmed_words = [stemmer.stem(word) for word in words]
    
    # Join the stemmed words back into a single string
    processed_text = ' '.join(stemmed_words)
    
    return processed_text

# Apply the preprocess_text function to the "text" column of your DataFrame
new_df["text"] = new_df["text"].apply(preprocess_text)

In [9]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(new_df['text'])

In [10]:
k = 2  # Number of clusters for binary output
kmeans = KMeans(n_clusters=k, random_state=42)
kmeans.fit(tfidf_matrix)

# Assign binary labels based on cluster centroids
centroid_labels = kmeans.predict(tfidf_matrix)
centroid_labels_mapping = {0: 0, 1: 1}  # Mapping cluster labels to binary labels
new_df['label'] = [centroid_labels_mapping[label] for label in centroid_labels]

In [11]:
new_df

,id,text,time,label
0,5,passion technolog,09-03-2017 22:00,0
1,5,frustrat work,05-06-2017 13:05,0
2,5,ram alon,04-08-2017 02:30,0
3,5,depress,08-09-2017 02:35,0
4,5,feel lone,9/14/2017 20:09,1
...,...,...,...,...
974,999,frighten ghost,3/19/2017 5:53,0
975,999,feel crazi,04-12-2017 19:25,1
976,999,feel proud friend like,12-09-2017 03:13,1
977,999,danger,2/13/2017 4:48,0


In [14]:
df['label']=new_df['label']

In [13]:
from sklearn.metrics import silhouette_score
silhouette_avg = silhouette_score(tfidf_matrix, kmeans.labels_)
print("Silhouette Score:", silhouette_avg)

Silhouette Score: 0.06276989362457683


In [15]:
df

,id,text,time,label
0,5,Passion about technologies,09-03-2017 22:00,0
1,5,Frustrated about work,05-06-2017 13:05,0
2,5,ram is alone,04-08-2017 02:30,0
3,5,am in depression,08-09-2017 02:35,0
4,5,Feeling lonely,9/14/2017 20:09,1
...,...,...,...,...
974,999,Frightened of ghost,3/19/2017 5:53,0
975,999,Feeling Crazy,04-12-2017 19:25,1
976,999,Feeling proud to have a friend like you,12-09-2017 03:13,1
977,999,I am in danger,2/13/2017 4:48,0


In [17]:
df.to_csv('labelled_df.csv')

In [18]:
train_df=pd.read_csv('dreaddit-train.csv')

In [21]:
merged_df = train_df.append(df[['text', 'label']], ignore_index=True)

C:\Users\upend\AppData\Local\Temp\ipykernel_17612\4176694564.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merged_df = train_df.append(df[['text', 'label']], ignore_index=True)


In [22]:
train_df.shape

(2838, 116)

In [25]:
merged_df.to_csv('improved_dataset.csv')